In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Importing the Libraries 


In [ ]:
# for importing and descriping
import pandas as pd
import numpy as np

# for visualization 
import matplotlib.pyplot as plt
import seaborn as sns

#for clustering 
from sklearn.cluster import AgglomerativeClustering , KMeans
from sklearn.metrics import silhouette_score 
#for Dendogram
import scipy.cluster.hierarchy as sch 

# Describing the dataset


In [ ]:
#Reading the mall dataset 
df=pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')

#Showing the first 10 rows  
df.head(10)

In [ ]:
#Showing the first 10 rows  
df.head(10)

In [ ]:
#Describing the shape of the dataset and the Descriptive statistics
print(df.shape) 
df.describe()

In [ ]:
print(df.median().astype(int))
d1= df.mode().head(1).drop('CustomerID', axis=1)
d1

In [ ]:
data = {
        'Descriptions':  ['Avarge', 'Most apparent ','Maximum value','Minimum value'],
        'Gender': [np.nan, d1['Gender'].values[0] , np.nan , np.nan] ,  
        'Age': [df['Age'].mean(), d1['Age'].values[0] ,df['Age'].max() , df['Age'].min()] ,  
        'Annual Income (k$)': [df['Annual Income (k$)'].mean(), d1['Annual Income (k$)'].values[0] ,df['Annual Income (k$)'].max() , df['Annual Income (k$)'].min()] ,
        'Spending Score (1-100)': [df['Spending Score (1-100)'].mean(), d1['Spending Score (1-100)'].values[0] ,df['Spending Score (1-100)'].max() , df['Spending Score (1-100)'].min()] 
        }
descriping_data = pd.DataFrame (data, columns = ['Descriptions','Gender','Age' , 'Annual Income (k$)' , 'Spending Score (1-100)'])
descriping_data

> The table shows the statistical description of the dataset ( the average value of the numerical data like the income and the score of spending in the mall,  maximum values, minimum values, and the most apparent values for each feature(column))

# Dealing with missing values and duplication


In [ ]:
#Missing values
df.isna().sum()

In [ ]:
#Duplicated values(duplicated rows)
df.duplicated().sum()

# Dealing with Outliers

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.figure(figsize=(25,10))
for i,col in enumerate(df.drop(columns=['CustomerID' , 'Gender']).columns):
    plt.subplot(2,4,i+1)
    sns.boxplot(df[col])

In [ ]:
def Outlier_Bounds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.25)
    quartile3 = dataframe[variable].quantile(0.75)
    #interquantile range 
    IQR = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * IQR
    low_limit = quartile1 - 1.5 * IQR
    return low_limit, up_limit

def has_Outliers(dataframe, variable):
    lower_limit, upper_limit = Outlier_Bounds(dataframe, variable)
    if dataframe[(dataframe[variable] < lower_limit) | (dataframe[variable] > upper_limit)].any(axis=None):
        print(variable, "yes")
    else :    
        print(variable, "no")

In [ ]:
for col in df.loc[:,df.columns!='Gender']:
    has_Outliers(df, col)

In [ ]:
#if Outliers exist ,then :
def replace_with_Bounds(dataframe, variable) :
    lower_limit, upper_limit = Outlier_Bounds(dataframe, variable)
    dataframe.loc[(dataframe[variable])<lower_limit ,variable]=lower_limit
    dataframe.loc[(dataframe[variable])>upper_limit ,variable]=upper_limit

In [ ]:
#replace Outliers with upper&lower limits (maximum and minimum )of the interquantile range 
for col in df.loc[:,df.columns!='Gender']:
    replace_with_Bounds(df, col)
    
for col in df.loc[:,df.columns!='Gender']:
    has_Outliers(df, col)

In [ ]:
plt.figure(figsize=(25,10))
for i,col in enumerate(df.drop(columns=['CustomerID' , 'Gender']).columns):
    plt.subplot(2,4,i+1)
    sns.boxplot(df[col])

# Visualization

In [ ]:
df1=df.drop(columns='CustomerID' ,axis=1)
df1

In [ ]:
print(df1['Gender'].value_counts())
plt.pie(df1['Gender'].value_counts() , autopct='%1.0f%%' , colors=['#d44000','#ff7a00'] , 
        labels=['Female','Male'] , explode =[0, 0.1] ,textprops=dict(color='w', fontsize=15 , fontweight='bold'))
plt.title('Gender', fontsize = 25 , fontweight='bold' );

**By looking at the plots which explain the distribution of Gender :**
>         we can see the rate of females shopping in the mall with a share of 56% is greater than
        males 44%. 
>         This shows that females are in the lead of shopping 

In [ ]:
plt.rcParams['figure.figsize'] = (20, 8)
plt.subplot(2,1,1)
sns.countplot(df1['Age'].astype('int') ,palette='YlOrBr_d')
plt.title('Age' , fontsize=20)
plt.subplot(2,1,2)
sns.distplot(df1['Age'].astype('int'), hist=False , color='#f9b208')
plt.axvline(x=32 , c='red' )


**the distribution of each Age Group in the Mall for clarity about the Visitor's Age Group :**
>         It can be seen that the number of visitors of the Mall is between the Age 18 and 67 . People 
>         at Age 32 are the Most 
>         Frequent Visitors , Ages from 23 to 40 are the most common age group for the mall . Older age 
>         groups have the lowest 
>         number of visitors, as people aged 55, 56, 69, and 64 are the least Presence in malls

In [ ]:
plt.rcParams['figure.figsize'] = (20, 8)
plt.subplot(2,1,1)
sns.countplot(df1['Annual Income (k$)'].astype('int') , palette='YlOrBr_d')
plt.title('Annual Income (k$)', fontsize=20)
plt.subplot(2,1,2)
sns.distplot(df1['Annual Income (k$)'], hist=False, color='#f9b208')

plt.axvline(x=48 , c='r')
plt.axvline(x=88 , c='r')

**The distribution of each Income level :** 
>         customers in the mall have much similar frequency with their Annual Income ranging from 15k 
>          US Dollars to 137K US Dollars. There are Customers in the Mall who have the higher Annual 
>          Income with 54k US Dollars or 78k US Dollars.       

In [ ]:
plt.rcParams['figure.figsize'] = (20, 8)
sns.countplot(df1['Spending Score (1-100)'].astype('int')  , palette='YlOrBr_d')
plt.title('Spending Score (1-100)', fontsize=20)


**The distribution of Spending Score :**
>     It is very Important to have a look and an idea about the Spending Score of the Customers  
>     Visiting the Mall.
>     Most of the customers Spending Scores range between 35 to 75. The mall provides most of the 
>     requirements of visitors as there are many customers having spending scores greater than 90.**

In [ ]:
plt.subplot(2,1,1)
sns.lineplot(x="Age", y="Annual Income (k$)" ,data=df1, hue='Gender'  , palette='YlOrBr_d')
plt.subplot(2,1,2)
sns.lineplot(x="Age",y="Spending Score (1-100)" ,data=df1, hue='Gender', palette='YlOrBr_d')

**The graph shows the relation between Age and Annual income and the relation between Age and 
    Spending scores according to the visitors Gender.
         The income line shows that the Almost Age group between 25 to 40 for men and 27 to 45  have 
         the most annual income, and the scores line shows that the Almost Age group between 20 to 35
         for men and 20 to 40 have the most scores.
         Which means that the most targeted age group for promotions and marketing is between
         the ages of 20 to 45 .**


In [ ]:
sns.pairplot(df1 ,hue="Gender" , palette='YlOrBr_d')

**The curve graph shows that although the men's group has the highest income than the women's, the 
    women's spending score is higher than the men's and the Age rate of women is the highest .
    A scatter chart showing the most targeted group for marketing, as we explained before.
    The attention will be on age and annual income more than a particular gender** 

In [ ]:
sns.catplot(x='Gender' , y= 'Annual Income (k$)' , data=df1 , kind='violin' , palette='YlOrBr_d')
#sns.catplot(x='Gender' , y= 'Annual Income (k$)' , data=df1 , kind='box' , palette='YlOrBr_d')

sns.catplot(x='Gender' , y= 'Spending Score (1-100)' , data=df1 , kind='violin' , palette='YlOrBr_d')
#sns.catplot(x='Gender' , y= 'Spending Score (1-100)' , data=df1 , kind='box' , palette='YlOrBr_d')


 **More Analysis between Gender , Spending Score and the Income .**
>     The most of males have an annual income between 45k Dollars to 80k Dollars while females have an 
>     annual income between 40k US Dollars to less than 80k US Dollars.     
>     The most of males have a spending score between 25 to 70 while females have a spending score 
>     between 35 to 75.
    
***That again points to that women are Shopping Leaders .This confirms our saying that whatever the
    increase in men's income, they are less to spend than women on shopping and buying***

In [ ]:
plt.rcParams['figure.figsize'] = (18, 8)
sns.heatmap(df1.corr(),annot=True ,cmap='YlOrBr_r' )

 **The Heat map shows the correlation between the different attributes of the Mall Customer 
    Segementation as the least correlated features is with the darker color.We can see that 
    attributes do not have good correlation among them**



In [ ]:
data=df1.drop(axis=1 ,columns= ['Gender','Age'])
data.head()

In [ ]:
data2=df1.drop(axis=1 ,columns= ['Gender','Annual Income (k$)'])
data2.head()

# Clustering Analysis:

### Heirarchical  Clustering :


In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
dendo=sch.dendrogram(sch.linkage(data,method='ward') , leaf_rotation=90 , leaf_font_size=5)
plt.ylabel('Euclidean distanse')
plt.xlabel('Customers')
plt.title('Dendogram1')
plt.axhline(y=200 , color='k')
#number of clusters = 5

plt.subplot(1,2,2)
dendo=sch.dendrogram(sch.linkage(data2,method='ward') , leaf_rotation=90 , leaf_font_size=5)
plt.ylabel('Euclidean distanse')
plt.xlabel('Customers')
plt.title('Dendogram2')
plt.axhline(y=150 , color='k')
#number of clusters = 4

In [ ]:
#from the dendogram ... the number of clusters = 5
cluster1_1=AgglomerativeClustering(n_clusters=5 , affinity='euclidean' , linkage='ward')
cluster1_1.fit_predict(data)

cluster1_2=AgglomerativeClustering(n_clusters=4 , affinity='euclidean' , linkage='ward')
cluster1_2.fit_predict(data2)

plt.figure(figsize=(15,8))

plt.subplot(1,2,1)
plt.scatter(data.loc[:,['Annual Income (k$)']], data.loc[:,['Spending Score (1-100)']], c=cluster1_1.labels_ )
plt.title('Customer Segmentation 1' , fontsize=15 , fontweight='bold')
plt.xlabel('Annual Income(k$)')
plt.ylabel('Spending Score(1-100')


plt.subplot(1,2,2)
plt.scatter(data2.loc[:,['Age']], data2.loc[:,['Spending Score (1-100)']], c=cluster1_2.labels_ )
plt.title('Customer Segmentation 2' , fontsize=15 , fontweight='bold')
plt.xlabel('Age)')
plt.ylabel('Spending Score(1-100')

**Customer Segmentation 1 gives us a clear insight about the different groubs of the customers in the Mall based on their Annual Income. 
There are Five groups of Customers namely Economizer (Purple), Careful (Yellow) ,
General (dark Green), Target (Blue),and Wasteful (light Green) based on their Annual Income and Spending Score which are the best features to determine the segments of a customer .**

**Customer Segmentation 2 also tells us about the different groubs of the customers in the Mall but this time based on their Age. 
There are Four different groups of Customers namely Usual (Green), Priority(Yellow) ,Target[Old] (Blue),and Target [Young] (Purple) based on their Age and Spending Score which are anther features to determine the segments of a customer.**

### K-means  Clustering :

In [ ]:
# Elbow method in k_means to specify the number of clusters
range_of_clusters1=range(1,11) # or 10 ----> k in Elbow method
wcss1=[]
for k in range_of_clusters1:
    km1_1=KMeans(n_clusters= k ,init='k-means++')
    km1_1.fit(data)
    # inertia_ for return wcss for the model
    wcss1.append(km1_1.inertia_)

wcss2=[]
for k in range_of_clusters1:
    km1_2=KMeans(n_clusters= k ,init='k-means++')
    km1_2.fit(data2)
    # inertia_ for return wcss for the model
    wcss2.append(km1_2.inertia_)

    
# Silhouette method in k_means to specify the number of clusters
range_of_clusters2=range(2,11)
scores1=[]
for k in range_of_clusters2 :
    km2_1=KMeans(n_clusters=k , init='k-means++' )
    km2_1.fit(data)
    scores1.append(silhouette_score(data , km2_1.labels_))
    
scores2=[]
for k in range_of_clusters2 :
    km2_2=KMeans(n_clusters=k , init='k-means++' )
    km2_2.fit(data2)
    scores2.append(silhouette_score(data2 , km2_2.labels_))
    

In [ ]:
#ploting
plt.figure(figsize=(30,10)) 

plt.subplot(2,2,1)
plt.plot(range_of_clusters1 , wcss1 , marker='o' )
plt.axvline(x=5 , c='r')
plt.title("Elbow method 1" , fontweight='bold')

plt.subplot(2,2,2)
plt.plot(range_of_clusters2 , scores1 , marker='o' )
plt.axvline(x=5 , c='r')
plt.title("Silhouette method 1" , fontweight='bold')

plt.subplot(2,2,3)
plt.plot(range_of_clusters1 , wcss2 , marker='o' )
plt.axvline(x=4 , c='r')
plt.title("Elbow method 2" , fontweight='bold')

plt.subplot(2,2,4)
plt.plot(range_of_clusters2 , scores2 , marker='o' )
plt.axvline(x=4 , c='r')
plt.title("Silhouette method 2" , fontweight='bold')


In [ ]:
#from the K-means methods (Elbow method ,Silhouette method) ... the number of clusters = 5
cluster2_1=KMeans(n_clusters=5 , init='k-means++')
cluster2_1.fit_predict(data)

cluster2_2=KMeans(n_clusters=4 , init='k-means++')
cluster2_2.fit_predict(data2)

plt.figure(figsize=(20,5)) 
plt.subplot(1,2,1)
plt.scatter(data.loc[:,['Annual Income (k$)']], data.loc[:,['Spending Score (1-100)']], c=cluster2_1.labels_  )
plt.scatter(cluster2_1.cluster_centers_[:,0], cluster2_1.cluster_centers_[:,1], c='r' )
plt.title('Customer Segmentation based on Annual Incomes' , fontsize=15 , fontweight='bold')
plt.xlabel('Annual Income(k$)')
plt.ylabel('Spending Score(1-100') 


plt.subplot(1,2,2)
plt.scatter(data2.loc[:,['Age']], data2.loc[:,['Spending Score (1-100)']], c=cluster2_2.labels_  )
plt.scatter(cluster2_2.cluster_centers_[:,0], cluster2_2.cluster_centers_[:,1], c='r' )
plt.title('Customer Segmentation based on Ages' , fontsize=15 , fontweight='bold')
plt.xlabel('Age')
plt.ylabel('Spending Score(1-100') 


**Again with anther Clustering analysis model.**
 
  **The Five different segments of Customers : Economizer (Purple), Careful (dark Green) ,General
  (Blue),Target (Yellow),Wasteful (light Green) ,and Cenetr points(Red) based on their Annual Income 
  and Spending Score .**
 

  **The Four different segments of Customers namely Usual (Blue), Priority(Purple) ,Target[Old] 
  (Yellow),Target [Young] (Green) ,and Cenetr points(Red) based on their Age and Spending Score.**

# **Insight** :

### It is necessary to give more attention to the target groups of customers in the young age group and with higher incomes and then other groups with higher spending scores in any age group  through marketing and making offers  more according to each segment and its interest 